In [1]:

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import random

def reduce_dataset(directory, delete_percentage=50):
    """
    Delete a specified percentage of images from the dataset directory.

    :param directory: Path to the directory containing the images.
    :param delete_percentage: Percentage of images to delete (e.g., 50 for 50%).
    """
    # Get a list of all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    # Calculate the number of files to delete
    num_files_to_delete = int(len(files) * delete_percentage / 100)

    # Randomly select files to delete
    files_to_delete = random.sample(files, num_files_to_delete)

    # Delete the selected files
    for file in files_to_delete:
        os.remove(os.path.join(directory, file))
        print(f"Deleted {file}")

    print(f"Deleted {num_files_to_delete} files out of {len(files)} total files.")

# Usage
dataset_directory = '../data/raw/img_align_celeba/'
reduce_dataset(dataset_directory, delete_percentage=80)


In [2]:
from src.models.model import build_sr_model
# Build the model

model = build_sr_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 up_sampling2d (UpSampling2  (None, 256, 256, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 up_sampling2d_1 (UpSamplin  (None, 512, 512, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 512, 512, 64)      36928 

In [3]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [4]:
from src.utils.utils import generate_batches, plot_history

image_directory = '../data/raw/img_align_celeba/'
batch_size = 16

# Use the generator function in your model's fit method
history = model.fit(
    generate_batches(image_directory, batch_size, ),
    steps_per_epoch=len(os.listdir(image_directory)) // batch_size,
    epochs=10
)

# Plot the training and validation loss to check for overfitting
plot_history(history)

Found 101300 images belonging to 1 classes.
Epoch 1/10
   8/6331 [..............................] - ETA: 9:15:10 - loss: 0.0771